In [40]:
# enviroment set-up
import os
import sys
sys.path.append("..")
sys.path.append("D:\\20120321_DHBW_AZUREML\\sato")
sys.path.append("D:\\20120321_DHBW_AZUREML\\sato\\model")
from os.path import join

# set env-var
os.environ['BASEPATH'] = 'D:\\20120321_DHBW_AZUREML\\sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dhbwmlc-ds12-v2/code/Users/svenlangenecker/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type78'


from model import datasets
from sklearn.preprocessing import LabelEncoder
from utils import get_valid_types
import copy
import torch
from torch.utils.data import ConcatDataset

In [50]:
label_enc = LabelEncoder()
label_enc.fit(get_valid_types(os.environ["TYPENAME"]))

LabelEncoder()

# Loading data in train_sherlock

In [51]:
import json
import pandas as pd
column_level_split_file_path = "D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_1_absolute_20.0.json"
#embclus_gen_training_data_path = "D:\\semantic_data_lake\\semantic_data_lake\\emb_clus\\knn_classifier\\out\\gen_training_data\\gen_training_data_1_1e-06_10_70_20.csv"
with open(column_level_split_file_path) as f:
    labeled_unlabeled_test_split_file = json.load(f)

#embclus_gen_training_data_path = "D:\\semantic_data_lake\\semantic_data_lake\\emb_clus\\knn_classifier\\out\\gen_training_data\\gen_training_data_1_1e-06_10_70_20.csv"
#df_embclus_training_data = pd.read_csv(embclus_gen_training_data_path)
#df_embclus_training_data["field_name"] = label_enc.transform(df_embclus_training_data["predicted_semantic_type"].tolist())
#df_embclus_training_data

# Loading data in train_CRF_LC

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from model import models_sherlock
from utils import name2dic

topic = "num-directstr_thr-0_tn-400"

if topic:
    topic_dim = int(name2dic(topic)['tn'])
else:
    topic_dim = None

# load single column model
# pre_trained_sherlock_loc = join(
#     os.environ['BASEPATH'], 'model', 'pre_trained_sherlock', os.environ["TYPENAME"])
# classifier = models_sherlock.build_sherlock(['char', 'rest', 'par', 'word'], num_classes=len(
#     get_valid_types(os.environ["TYPENAME"])), topic_dim=topic_dim, dropout_ratio=0.35).to(device)

# classifier.load_state_dict(torch.load(
#             join(pre_trained_sherlock_loc, "all_None.pt"), map_location=device))

In [53]:
label_enc = LabelEncoder()
label_enc.fit(get_valid_types(os.environ["TYPENAME"]))
print(os.environ["TYPENAME"])

whole_corpus = datasets.TableFeatures("public_bi_benchmark",['char', 'rest', 'par', 'word'], topic_feature=topic, label_enc=label_enc, id_filter=None, max_col_count=100)

type78
D:\20120321_DHBW_AZUREML\sato\tmp\public_bi_benchmark_type78_header_valid.pkl pickle file found, loading...
public_bi_benchmark_type78_header_valid Load complete. Time 0.01585865020751953
Total data preparation time: 0.15108585357666016


In [54]:
whole_corpus.df_header.loc["CityMaxCapita+CityMaxCapita_1"]

field_list       [1, 2, 10, 14, 20, 22, 23, 29, 30]
field_names    [70, 12, 37, 43, 62, 22, 70, 46, 46]
Name: CityMaxCapita+CityMaxCapita_1, dtype: object

In [73]:
train_list = []
#train_1 = 
#train = copy.copy(whole_corpus).set_filter_for_column_level_split(labeled_unlabeled_test_split_file["labeled1"]+labeled_unlabeled_test_split_file["unlabeled"])
train = copy.copy(whole_corpus).set_filter_for_column_level_split(['CityMaxCapita_1+column_1', "CityMaxCapita_1+column_2"])

train_list.append(train)

training_data = ConcatDataset(train_list)

Total time for splitting on column-level: 0.003998994827270508


In [74]:
train.df_header.loc["CityMaxCapita+CityMaxCapita_1"]

field_list       [1, 2]
field_names    [70, 12]
Name: CityMaxCapita+CityMaxCapita_1, dtype: object

In [76]:
train = train.overwrite_col_labels_from_embclus(pd.DataFrame({"table":["CityMaxCapita_1","CityMaxCapita_1"], "column":["column_1","column_2"], "dataset_id":["CityMaxCapita_1+column_1","CityMaxCapita_1+column_2"], "predicted_semantic_type":["country", "country"]}))

In [77]:
train.df_header.loc["CityMaxCapita+CityMaxCapita_1"]

field_list       [1, 2]
field_names    [22, 22]
Name: CityMaxCapita+CityMaxCapita_1, dtype: object

In [68]:
sherlock_train = train.to_col()

Time used to convert to SherlockDataset (column features) 0.0019998550415039062


In [78]:
train.data_dic["rest"]

{'CityMaxCapita+CityMaxCapita_1': tensor([[ 7.5694e+00,  5.0400e-01,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  9.2640e+00,  3.7617e+00,  0.0000e+00,  0.0000e+00,
           2.0780e+00,  6.8842e-01,  1.0000e+03,  1.0000e+00,  1.0000e+00,
           1.1165e+01,  2.1946e+01,  3.0000e+00,  2.8000e+01,  1.2000e+01,
           1.1165e+04, -2.5186e-01, -4.1184e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 7.7183e+00,  5.2000e-01,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  7.6910e+00,  3.1559e+00,  0.0000e+00,  0.0000e+00,
           1.2660e+00,  4.8088e-01,  1.0000e+03,  1.0000e+00,  1.0000e+00,
           7.9640e+00,  1.2093e+01,  3.0000e+00,  2.8000e+01,  8.0000e+00,
           7.9640e+03,  1.3677e+00,  6.4430e-01,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00]])}

In [79]:
sherlock_train.__getitem__(1)["data"]["rest"]

tensor([7.7183e+00, 5.2000e-01, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        7.6910e+00, 3.1559e+00, 0.0000e+00, 0.0000e+00, 1.2660e+00, 4.8088e-01,
        1.0000e+03, 1.0000e+00, 1.0000e+00, 7.9640e+00, 1.2093e+01, 3.0000e+00,
        2.8000e+01, 8.0000e+00, 7.9640e+03, 1.3677e+00, 6.4430e-01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])

In [8]:
#train.df_header.loc["CityMaxCapita+CityMaxCapita_1"]
train.df_header
df_header = copy.copy(train.df_header)
#df_header

In [75]:
test_list = []
new_data = ["test_id", [0,1], [67,77]]
test_list.append(new_data)
pd.DataFrame(test_list, columns=["table_id", "field_list", "field_names"])

,table_id,field_list,field_names
0,test_id,"[0, 1]","[67, 77]"


In [61]:
%%time
df_header = copy.copy(train.df_header).values.tolist()
for i in range(1000):
    df_header.append(df_header[0])
df = pd.DataFrame(df_header, columns=["1","2"])

Wall time: 3.01 ms


In [62]:
%%time
df_header = copy.copy(train.df_header)
for i in range(1000):
    df_header = pd.concat([df_header, df_header.iloc[1]], ignore_index=True)

Wall time: 2.31 s


In [63]:
%%time
df_header = copy.copy(train.df_header)
for i in range(1000):
    df_header = df_header.append(df_header.iloc[1], ignore_index=True)

Wall time: 1.36 s


In [10]:
train = train.overwrite_col_labels_from_embclus(df_embclus_training_data)

NameError: name 'df_embclus_training_data' is not defined

In [10]:
df_header.loc["CityMaxCapita+CityMaxCapita_1"]

NameError: name 'df_header' is not defined

In [11]:
train.df_header.loc["CityMaxCapita+CityMaxCapita_1"]

field_list       [1, 2, 10, 14, 20, 22, 23, 30]
field_names    [70, 12, 37, 43, 62, 22, 70, 46]
Name: CityMaxCapita+CityMaxCapita_1, dtype: object

In [ ]:
for index, row in df_embclus_training_data.iterrows():
    # if index > 2:
    #     break
    #print(index)
    dataset_id = row["table"]
    locator = dataset_id.split("_")[0]
    table_id = locator+"+"+dataset_id
    column_id = int(row["column"].split("_")[1])
    
    field_list = eval(df_header.loc[table_id]["field_list"])
    field_names = eval(df_header.loc[table_id]["field_names"])

    # selection in df_header
    #print(df_header.loc[table_id])
    #print(field_list.index(column_id))
    searched_index = field_list.index(column_id)

    # override field_names with the predicted semantic type
    field_names[searched_index] = row["field_name"]

    #print(field_names)

    df_header.loc[table_id]["field_names"] = str(field_names)

In [ ]:
df_header.loc["CityMaxCapita+CityMaxCapita_1"]

In [ ]:
train_list = []
train = copy.copy(whole_corpus).set_filter([
    "CommonGovernment+CommonGovernment_10"
])

test = copy.copy(whole_corpus).set_filter([
    #"MLB+MLB_1",
    "MLB+MLB_46"
])

train_list.append(train)

training_data = ConcatDataset(train_list)

In [ ]:
batch_size = 1
n_worker = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
training = datasets.generate_batches(training_data,
                                                 batch_size=batch_size,
                                                 shuffle=False,
                                                 drop_last=True,
                                                 device=device,
                                                 n_workers=n_worker)

from tqdm import tqdm
import numpy as np
import numpy.ma as ma
from sklearn.metrics import classification_report
from torchcrf import CRF
y_pred, y_true = [], []

classifier.eval()

training_iter = tqdm(training, desc="Training")
for table_batch, label_batch, mask_batch in training_iter:
    for f_g in table_batch:
        table_batch[f_g] = table_batch[f_g].view(batch_size * 100, -1)

    pred_scores = classifier.predict(table_batch)
    #print(pred_scores[0])
    pred = torch.argmax(pred_scores, dim=1).cpu().numpy()
    #print(pred)

    labels = label_batch.view(-1).cpu().numpy()
    masks = mask_batch.view(-1).cpu().numpy()
    invert_masks = np.invert(masks == 1)
    #print(labels)

    y_pred.extend(ma.array(pred, mask=invert_masks).compressed())
    y_true.extend(ma.array(labels, mask=invert_masks).compressed())

    print("Predictions: ",y_pred)
    print("True types: ",y_true)
    #print(classification_report(y_true, y_pred))

    emissions = classifier(table_batch).view(
                    batch_size, 100, -1).to(device)
    #print("Emissions: ",emissions)

    model = CRF(len(get_valid_types(os.environ["TYPENAME"])), batch_first=True).to(device)
    # load pretrained model
    model_loc = join(os.environ['BASEPATH'], 'model',
                        'pre_trained_CRF', os.environ["TYPENAME"])
    loaded_params = torch.load(
        join(model_loc, "CRF+LDA_pathL.pt"), map_location=device)
    model.load_state_dict(loaded_params['CRF_model'])
    model.eval()

    pred = model.decode(emissions, mask_batch)
    print("SATO Prediction:")
    print(pred)

    print(classification_report(y_true, pred[0]))


In [ ]:
import numpy.ma as ma
labels = label_batch.view(-1).cpu().numpy()
masks = mask_batch.view(-1).cpu().numpy()
invert_masks = np.invert(masks == 1)

ma.array(labels, mask=invert_masks).compressed()

In [ ]:
mask_batch[0][0] = 1
mask_batch[0][1] = 1
mask_batch[0][2] = 0
mask_batch[0][3] = 0
mask_batch[0][4] = 0
print(mask_batch)
print(mask_batch[0].all())
print(mask_batch[:,1])

In [ ]:
%run test_loading_dataset.py

# look in table topic extraction

In [ ]:
from extract.helpers.read_raw_data import get_filtered_dfs_by_corpus
from extract.helpers.utils import valid_header_iter_gen


corpus = "public_bi_benchmark"
TYPENAME = "type78"
header_name = "{}_{}_header_valid.csv".format(corpus, TYPENAME)
header_iter = valid_header_iter_gen(header_name)

print(header_iter)
#raw_df_iter = get_filtered_dfs_by_corpus["publicbibenchmark"](header_iter)

for df_features in header_iter:
    print(df_features)

In [30]:
import pandas as pd
import ast
import numpy as np

sherlock_feature_groups = ['char', 'word', 'par', 'rest']
other_feature_groups = ['topic']
column_level_id_filter = [
    "MLB_1+column_37",
    "MLB_1+column_45", 
    "MLB_46+column_52",
    "MLB_46+column_58"]

column_level_id_filter = labeled_unlabeled_test_split_file["test20.0"]+labeled_unlabeled_test_split_file["unlabeled"]

df_dic = {}

for training_set in column_level_id_filter:
    dataset_id = training_set.split("+")[0]
    # this locator is only suitable for public_bi corpus
    locator = dataset_id.split("_")[0]
    table_id = locator+"+"+dataset_id
    column_id = int(training_set.split("+")[1].split("_")[1])
    if table_id not in df_dic.keys():
        df_dic[table_id] = [column_id]
    elif table_id in df_dic.keys():
        df_dic[table_id].append(column_id)

#print(df_dic)

# generate new df_header
df_header_new = pd.DataFrame(
    {"table_id": [], "field_list": [], "field_names": []})
## manipulate data_dic
old_data_dic = whole_corpus.data_dic
new_data_dic = {}
for f_g in old_data_dic.keys():
    new_data_dic[f_g] = {}
for f_g in sherlock_feature_groups:
    new_data_dic[f_g] = {}
for f_g in other_feature_groups:
    new_data_dic[f_g] = {}
old_data_dic = whole_corpus.data_dic

for table_id in df_dic.keys():
    #print(table_id)
    # sort every field_list
    df_dic[table_id] = sorted(df_dic[table_id])

    old_field_list = np.array(eval(
        whole_corpus.df_header.loc[table_id]["field_list"]))
    #print(old_field_list)

    old_field_names = np.array(eval(
        whole_corpus.df_header.loc[table_id]["field_names"]))

    new_field_list = df_dic[table_id]
    mask_fields = np.array([x in new_field_list for x in old_field_list])
    new_field_names = old_field_names[mask_fields].tolist()

    df_header_new = df_header_new.append(pd.DataFrame({"table_id": [table_id], "field_list": [
                                            str(new_field_list)], "field_names": [str(new_field_names)]}), ignore_index=True)

    
    for f_g in old_data_dic.keys():
        new_data_dic[f_g][table_id] = old_data_dic[f_g][table_id][np.argwhere(mask_fields == True).flatten()]
    for f_g in sherlock_feature_groups:
        new_data_dic[f_g][table_id] = old_data_dic[f_g][table_id][np.argwhere(
            mask_fields == True).flatten()]
    for f_g in other_feature_groups:
        new_data_dic[f_g][table_id] = old_data_dic[f_g][table_id]
    
    
df_header_new = df_header_new.set_index("table_id")


In [ ]:
new_data_dic["rest"]

# Building NN with different number of types

In [3]:
from models_sherlock import FeatureEncoder, SherlockClassifier, build_sherlock
import torch
from os.path import join

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sherlock_feature_groups = ['char', 'rest', 'par', 'word']


classifier = build_sherlock(sherlock_feature_groups, num_classes=78, topic_dim=400, dropout_ratio=0.35).to(device)

model_loc = join(os.environ['BASEPATH'],
                         'model', 'pre_trained_sherlock', "type78")
pretrained_shelock_path = f"all_None.pt"

classifier.load_state_dict(torch.load(join(model_loc,pretrained_shelock_path), map_location=device))
print(classifier)

SherlockClassifier(
  (FeatureEncoder_char): FeatureEncoder(
    (bn1): BatchNorm1d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear1): Linear(in_features=960, out_features=300, bias=True)
    (relu1): ReLU()
    (dp1): Dropout(p=0.35, inplace=False)
    (linear2): Linear(in_features=300, out_features=300, bias=True)
    (relu2): ReLU()
  )
  (FeatureEncoder_rest): FeatureEncoder(
    (bn1): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear1): Linear(in_features=27, out_features=27, bias=True)
    (relu1): ReLU()
    (dp1): Dropout(p=0.35, inplace=False)
    (linear2): Linear(in_features=27, out_features=27, bias=True)
    (relu2): ReLU()
  )
  (FeatureEncoder_par): FeatureEncoder(
    (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear1): Linear(in_features=400, out_features=400, bias=True)
    (relu1): ReLU()
    (dp1): Dropout(p=0.35, inplace=False)
    (linear2)

In [5]:
print(classifier.linear3.in_features)

500


In [5]:
classifier.linear3.out_features = 255
print(classifier.linear3)
print(classifier.linear3.weight.data)
print(len(classifier.linear3.weight.data))

Linear(in_features=500, out_features=255, bias=True)
tensor([[-0.0928,  0.0366,  0.0606,  ...,  0.0746,  0.0451,  0.0076],
        [-0.0411, -0.0170,  0.0441,  ..., -0.0745, -0.0485, -0.0397],
        [ 0.0325, -0.0013, -0.0041,  ...,  0.0792,  0.0886, -0.0688],
        ...,
        [-0.0868, -0.0144,  0.0711,  ..., -0.0072, -0.1660, -0.0214],
        [ 0.0217,  0.0364, -0.0823,  ..., -0.0953, -0.0105, -0.0732],
        [ 0.0558, -0.0606,  0.0925,  ..., -0.0960, -0.0182, -0.0405]])
78


In [1]:
import torch.nn as nn

classifier.linear3 = nn.Linear(500, 255)
print(classifier.linear3.weight.data)
print(len(classifier.linear3.weight.data))

NameError: name 'classifier' is not defined

In [ ]:
# retrain sherlock and check the las layer 
import numpy as np
import os

# set env-var
os.environ['BASEPATH'] = 'D:\\20120321_DHBW_AZUREML\\sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dhbwmlc-ds12-v2/code/Users/svenlangenecker/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type78'

percent = 50
# if index != 0:
#     continue
comment = f"labeled{percent}_unlabeled{100-20-percent}_test{20}_255types"
column_level_split_file_path = f"D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_{percent}_{100-20-percent}_20.json"
pretrained_shelock_path = f"sherlock_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}_255types.pt"
pretrained_CRF_LDA_path = f"CRF+LDA_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}.pt"

# retrain sherlock
%run ../model/train_sherlock.py -c ../model/params/publicbi/sherlock_retrain.txt  --comment {comment} --column_level_split_file_path {column_level_split_file_path}

In [ ]:
pretrained_shelock_path = f"sherlock_retrain_labeled{percent}_unlabeled{100-20-percent}_test{20}_255types.pt"

%run ../model/train_sherlock.py -c ../model/params/publicbi/sherlock_eval.txt --model_list {pretrained_shelock_path} --comment "sherlock_baseline_column_level_split_test20" --column_level_split_file_path "D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_10_70_20.json"


In [10]:
from tqdm import tqdm
import numpy as np
import numpy.ma as ma
from sklearn.metrics import classification_report
from torchcrf import CRF

model = CRF(len(get_valid_types(os.environ["TYPENAME"])), batch_first=True).to(device)

In [13]:
model.start_transitions.data

tensor([-0.0239, -0.0149, -0.0826, -0.0511, -0.0959, -0.0794, -0.0002,  0.0272,
        -0.0487, -0.0631,  0.0808,  0.0082,  0.0215, -0.0863, -0.0891,  0.0871,
        -0.0959,  0.0727, -0.0038,  0.0935,  0.0261, -0.0030,  0.0183, -0.0093,
         0.0455,  0.0823,  0.0944, -0.0145,  0.0942,  0.0691, -0.0679, -0.0374,
        -0.0873, -0.0450, -0.0352,  0.0315,  0.0378, -0.0441, -0.0358, -0.0855,
        -0.0642, -0.0608, -0.0667, -0.0037,  0.0939, -0.0894, -0.0773,  0.0999,
         0.0339, -0.0458, -0.0915, -0.0563,  0.0210, -0.0369,  0.0050, -0.0677,
         0.0302, -0.0014, -0.0966, -0.0502,  0.0218, -0.0035,  0.0945, -0.0549,
        -0.0343,  0.0696, -0.0005, -0.0310,  0.0870, -0.0132,  0.0915,  0.0084,
        -0.0884,  0.0759,  0.0126,  0.0129,  0.0783,  0.0970])

In [17]:
model.transitions.data

tensor([[ 0.0980, -0.0819, -0.0186,  ...,  0.0923, -0.0348,  0.0196],
        [ 0.0367, -0.0308, -0.0216,  ...,  0.0458, -0.0933,  0.0820],
        [-0.0808, -0.0673,  0.0429,  ...,  0.0475, -0.0833,  0.0241],
        ...,
        [ 0.0523,  0.0672, -0.0991,  ...,  0.0986, -0.0761, -0.0089],
        [ 0.0313,  0.0764,  0.0594,  ...,  0.0779,  0.0865,  0.0542],
        [ 0.0740,  0.0074, -0.0683,  ...,  0.0216,  0.0247,  0.0993]])

In [42]:
test_labels = ["cat", "dog"]
label_enc_test = LabelEncoder()
label_enc_test.fit(test_labels)

print(label_enc_test.inverse_transform([0,1]))

['cat' 'dog']


In [43]:
test_labels.append("bird")
test_labels.append("zebra")
test_labels.append("aligator")
print(test_labels)

label_enc_test = LabelEncoder()
label_enc_test.fit(test_labels)

print(label_enc_test.inverse_transform([0,1,2,3,4]))

['cat', 'dog', 'bird', 'zebra', 'aligator']
['aligator' 'bird' 'cat' 'dog' 'zebra']


In [44]:
label_enc_test.classes_

array(['aligator', 'bird', 'cat', 'dog', 'zebra'], dtype='<U8')